# FastAPI Real-Time API with WebSockets

This notebook demonstrates how to build a **real-time API** using FastAPI and **WebSockets**. By the end of this tutorial, you will:
1. Understand the basics of WebSockets and how they differ from HTTP.
2. Set up a FastAPI app with WebSocket support.
3. Create a WebSocket endpoint for real-time communication.
4. Handle WebSocket connections and disconnections gracefully.
5. Test the WebSocket API using a Python client.

---

## 1. Understanding WebSockets

### What are WebSockets?
WebSockets provide a **full-duplex communication channel** over a single, long-lived connection between a client and a server. Unlike HTTP, which follows a request-response model, WebSockets allow **real-time, bidirectional communication**.

### Key Features of WebSockets:
- **Persistent Connection**: The connection remains open until explicitly closed by either the client or the server.
- **Low Latency**: Messages are sent and received instantly, making WebSockets ideal for real-time applications.
- **Bidirectional Communication**: Both the client and server can send messages at any time.

### Use Cases for WebSockets:
    - Chat applications
    - Live notifications
    - Real-time dashboards
    - Multiplayer online games

## 2. Connect to the FastAPI Kernel

Before starting, ensure you're using the correct kernel (`fastapi-env`) that has all the required dependencies installed.

### Instructions to Connect to the Kernel:
1. **Check Available Kernels**:
   - Click on the kernel name in the top-right corner of the notebook (e.g., `Python 3`).
   - Select **`Python (FastAPI)`** from the dropdown menu.

2. **Install Missing Dependencies**:
   - Open the notebook **fastapi-quickstart.ipynb** and follow the environment/kernel installation steps.
   - Restart the Jupyter kernel and select the `SilverAIWolf (FastAPI)` kernel.

In [ ]:
!source ./fastapi-env/bin/activate && pip install -q websockets

In [ ]:
from helper import FastAPIServer, clean_tutorial
import asyncio
import nest_asyncio
import websockets

## 3. Set Up the FastAPI App

First, let's create a basic FastAPI app with WebSocket support.

**Explanation**:
  - **`websocket.accept()`**: Accepts the WebSocket connection from the client.
  - **`websocket.receive_text()`**: Waits for a text message from the client.
  - **`websocket.send_text()`**: Sends a text message back to the client.
  - **`WebSocketDisconnect`**: This exception is raised when the client disconnects. It allows you to handle the disconnection gracefully (e.g., clean up resources or log the event).

In [ ]:
%%writefile main.py
from fastapi import FastAPI, WebSocket
from fastapi import WebSocket, WebSocketDisconnect

# Create an instance of the FastAPI class
app = FastAPI()

# WebSocket endpoint
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    try:
        while True:
            # Receive a message from the client
            data = await websocket.receive_text()
            print(f"Received message: {data}")
            
            # Send a response back to the client
            await websocket.send_text(f"Message received: {data}")
    except WebSocketDisconnect:
        print("Client disconnected")
    except Exception as e:
        print(f"Error in WebSocket handler: {e}")
        import traceback
        traceback.print_exc()  # Print the full traceback

## 4. Run the FastAPI Server

To run the FastAPI server.

The helper class method `FastAPIServer(port).run()` uses the following command: 
```bash
uvicorn main:app --reload --port 8081
```

**Explanation**:
  - `uvicorn` is an ASGI server used to run FastAPI apps.
  - `main` refers to the filename (`main.py`).
  - `app` refers to the FastAPI instance.
  - `--reload` enables auto-reloading during development.
  - `--port 8081` specifies the port to run the server on.

In [ ]:
port = 8081
fastapi = FastAPIServer(port)
fastapi.run()

## 5. Test the WebSocket API

Let's test the WebSocket API using a Python client. But since we are running an event loop already by using the jupyter notebook, we will have to apply nest_asyncio using the following command:

In [ ]:
nest_asyncio.apply()

**Explanation**:
  - The `websockets` library is used to create a WebSocket client.
  - The client connects to the WebSocket server at `ws://localhost:{port}/ws`.
  - It sends a message (`"Hello, WebSocket!"`) and waits for a response.

In [ ]:
async def test_websocket(port):
    uri = f"ws://localhost:{port}/ws"
    async with websockets.connect(uri) as websocket:
        # Send a message to the server
        await websocket.send("Hello, WebSocket!")
        
        # Receive a response from the server
        response = await websocket.recv()
        print(response)

# Run the WebSocket client
asyncio.get_event_loop().run_until_complete(test_websocket(port))

## 6. Build a Simple Real-Time Chat Application

Let's extend the WebSocket API to build a simple real-time chat application.

**Explanation**:
  - The `ConnectionManager` class manages active WebSocket connections.
  - `manager.connect()` adds a new connection to the list of active connections.
  - `manager.disconnect()` removes a connection when the client disconnects.
  - `manager.broadcast()` sends a message to all connected clients.

In [ ]:
%%writefile main.py
from fastapi import FastAPI, WebSocket
from fastapi import WebSocket, WebSocketDisconnect

# Create an instance of the FastAPI class
app = FastAPI()

class ConnectionManager:
    def __init__(self):
        self.active_connections = []

    async def connect(self, websocket: WebSocket):
        await websocket.accept()
        self.active_connections.append(websocket)

    def disconnect(self, websocket: WebSocket):
        self.active_connections.remove(websocket)

    async def broadcast(self, message: str):
        for connection in self.active_connections:
            await connection.send_text(message)

# Create an instance of the ConnectionManager
manager = ConnectionManager()

# WebSocket endpoint
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    try:
        while True:
            # Receive a message from the client
            data = await websocket.receive_text()
            print(f"Received message: {data}")
            
            # Send a response back to the client
            await websocket.send_text(f"Message received: {data}")
    except WebSocketDisconnect:
        print("Client disconnected")
    except Exception as e:
        print(f"Error in WebSocket handler: {e}")
        import traceback
        traceback.print_exc()  # Print the full traceback

@app.websocket("/ws/chat")
async def websocket_chat_endpoint(websocket: WebSocket):
    await manager.connect(websocket)
    try:
        while True:
            # Receive a message from the client
            data = await websocket.receive_text()
            print(f"Received message: {data}")
            
            # Broadcast the message to all connected clients
            await manager.broadcast(f"Sorry I can't talk because I am not a Generative Pre-trained Transformer. But soon you will make me smarter!")
    except WebSocketDisconnect:
        manager.disconnect(websocket)
        await manager.broadcast("A user has left the chat.")
    except Exception as e:
        print(f"Error in WebSocket handler: {e}")

## 7. Test the Chat Application

To test the chat application, you can use multiple WebSocket clients or a front-end interface. Here's an example using a Python client:
**Explanation**:
  - The chat client connects to the WebSocket server at `ws://localhost:{port}/ws/chat`.
  - It sends a message and listens for broadcasted messages.

In [ ]:
async def chat_client(port):
    uri = f"ws://localhost:{port}/ws/chat"
    async with websockets.connect(uri) as websocket:
        print("Connected to the chat server. Type 'exit' to quit.")
        while True:
            # Prompt the user for input
            message = input("You: ")
            
            # Exit the loop if the user types 'exit'
            if message.lower() == "exit":
                print("Exiting chat...")
                break
            
            # Send the message to the server
            await websocket.send(message)
            
            # Receive a response from the server
            response = await websocket.recv()
            print(f"Server: {response}")

# Run the chat client
asyncio.get_event_loop().run_until_complete(chat_client(port))

## Recap

In this notebook, you:
1. Learned about WebSockets and their use cases.
2. Set up a FastAPI app with WebSocket support.
3. Created a WebSocket endpoint for real-time communication.
4. Handled WebSocket connections and disconnections gracefully.
5. Built a simple real-time chat application.
6. Tested the chat application using a Python client.

## Next Steps

1. **Integrate with Front-End Frameworks**:
   - Build a front-end interface using frameworks like React, Vue.js, or Angular to interact with the WebSocket API.

2. **Scale the Application**:
   - Use tools like Redis or Kafka to handle high volumes of messages and scale the chat application.

3. **Add Authentication**:
   - Implement user authentication to secure the WebSocket connection and personalize chat messages.

4. **Explore Advanced Features**:
   - Add features like typing indicators, message history, and file sharing to enhance the chat application.

5. **Move to Production**:
   - Deploy the FastAPI app using tools like Docker, Kubernetes, or cloud platforms (e.g., AWS, GCP, Azure).

6. **Learn More**:
   - Explore advanced WebSocket integration techniques in the next notebook: **`fastapi-websockets-integration.ipynb`**.

# Optional Cleaning

In [ ]:
clean_tutorial()